# <center>  Model Creat Dataframe CAPITAL IMMOBILIER [KP0]<br/></center>
<a name="TOP"></a>
- dev : nyx
- Fichier CSV :  Capital_immobilier_csv/Capital_immobilier_csv/capitalimmobilier.csv</a>
    
# Description Fichiers : 
Transaction immmobilieres sur la communes 
sur 2 périodes P1 [2014;2017] & P2 [2018;2021]
avec pour chaque periode :
- prix total des transaction (Pri)
- Surface totale des transactions (Surf)
- Surface totale des tarrainss (SurfTer)
- Nombre de transactions (transac)
- Prix m² moyen (prim2)

# Synthese des Actions :
Plus de 1000 communes non renseignees flag 1 pour BL_bug
creation de lignes avec que des 0
  
**<u>MENU</u>** 
- <a href="#Import">P0 : Import CSV</a>
- <a href="#EDA">P1 : Mini EDA</a>
- <a href="#Nan">P2 : Traitement des Nan</a>
- <a href="#TRANSCO">P3 : Filtres et/ou calculs</a>
- <a href="#EXPORT">P4 : EXPORT (/DATA/EXPORT_CSV/final_xxxxxxx.csv)</a>





PREFIXE FICHIER : KP0
Structure :

    0 dep
    1 nomdep
    2 codecommune
    3 nomcommune
    4 KP0_Pri_P1 
    5 KP0_Surf_P1
    6 KP0_SurfTer_P1
    7 KP0_transac_P1
    8 KP0_prim2_P1
    9 KP0_Pri_P2
    10 KP0_Surf_P2
    11 KP0_SurfTer_P2
    12 KP0_transac_P2
    13 KP0_prim2_P2
    14 DBL_bug (flag à 1 poour les lignes ajoutées avec des 0)



In [43]:
import warnings
warnings.filterwarnings("ignore")

import os
import zipfile
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

## <a name="Import"></a>Partie 0 : Import CSV&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <a href="#TOP">&uarr;</a>

In [44]:
def df_creat(filename, df_dataframe):  
    df = df_dataframe[df_dataframe['filename'] == filename]
    if len(df) == 1:
        row = df.iloc[0]
        file = row['path']
        if os.path.exists(file):
            print(file)
            df_from_csv = pd.read_csv(file, encoding='utf-8', low_memory=False)
            return df_from_csv
        else:
            print('FICHIER NON TROUVE', file)
            return pd.DataFrame()
    else:
        print(f"filename: '{filename}' return size: {len(df)}")
        return pd.DataFrame()

In [45]:
csv_file_path = 'df_filtred.csv'
# Use pandas to read the CSV file into a DataFrame with UTF-8 encoding
df_cat = pd.read_csv(csv_file_path, encoding='utf-8')

In [46]:
df_cat.head(1)

,category,filename,comment,size,depart,district,commune,canton,yyyy,path,codecommune
0,Taille_agglo_commune_csv,codescommunes1999,codes,0,0,0,1,0,1999,../../ProjetCommun/DATA_SOURCES/HCP/Taille_agg...,1


In [48]:
df = df_creat('capitalimmobilier', df_cat)

../../ProjetCommun/DATA_SOURCES/HCP/Capital_immobilier_csv/Capital_immobilier_csv/capitalimmobilier.csv


## <a name="EDA"></a>Partie 1 : Mini EDA &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <a href="#TOP">&uarr;</a>

In [49]:
df.shape

(5455409, 14)

In [50]:
df['tmp']=df['prixbien']

In [51]:
df.head(5)

,annee,dep,nomdep,codecommune,nomcommune,plm,prixbien,prixm2,surface,surfaceterrain,codemaisonappartement,datemutation,novoie,voie,tmp
0,2014,01,AIN,01001,ABERGEMENT-CLEMENCIAT,0,40000.0,370.37036,108,1005,1,2014-01-30,6.0,LES MURIERS,40000.0
1,2014,01,AIN,01001,ABERGEMENT-CLEMENCIAT,0,93000.0,861.11108,108,1005,1,2014-01-27,6.0,LES MURIERS,93000.0
2,2014,01,AIN,01001,ABERGEMENT-CLEMENCIAT,0,134750.0,2173.38720,62,523,1,2014-02-20,5001.0,MUNETVILLE,134750.0
3,2014,01,AIN,01001,ABERGEMENT-CLEMENCIAT,0,186820.0,1569.91600,119,1087,1,2014-07-03,5.0,LES MURIERS,186820.0
4,2014,01,AIN,01001,ABERGEMENT-CLEMENCIAT,0,193000.0,1873.78640,103,833,1,2014-07-07,5172.0,MUNETVILLE,193000.0


In [52]:
for c in df : 
    print (c)
    

annee
dep
nomdep
codecommune
nomcommune
plm
prixbien
prixm2
surface
surfaceterrain
codemaisonappartement
datemutation
novoie
voie
tmp


In [53]:
df['annee'].min().min(),df['annee'].max().max()

(2014, 2021)

## Creation 2 periodes

In [54]:
# SPLIT 2 periodes fonctions des elelections
df1 = df[df['annee'] < 2018]
df2 = df[df['annee'] >= 2018]

In [55]:
df1.shape,df2.shape,

((2767929, 15), (2687480, 15))

In [56]:
df1.head(2)

,annee,dep,nomdep,codecommune,nomcommune,plm,prixbien,prixm2,surface,surfaceterrain,codemaisonappartement,datemutation,novoie,voie,tmp
0,2014,01,AIN,01001,ABERGEMENT-CLEMENCIAT,0,40000.0,370.37036,108,1005,1,2014-01-30,6.0,LES MURIERS,40000.0
1,2014,01,AIN,01001,ABERGEMENT-CLEMENCIAT,0,93000.0,861.11108,108,1005,1,2014-01-27,6.0,LES MURIERS,93000.0


In [57]:
df_group1 = df1.groupby(['dep', 'nomdep', 'codecommune', 'nomcommune']).agg({
    'prixbien': 'sum',
    'surface': 'sum',
    'surfaceterrain': 'sum',
    'tmp': 'count'
}).reset_index()

In [58]:
df_group1.head(2)

,dep,nomdep,codecommune,nomcommune,prixbien,surface,surfaceterrain,tmp
0,01,AIN,01001,ABERGEMENT-CLEMENCIAT,5190811.33,2960,29249,28
1,01,AIN,01002,ABERGEMENT-DE-VAREY,2442260.00,2091,7040,21


In [59]:
df_group1['KP0_prim2_P1'] = df_group1['prixbien'] / df_group1['surface']
df_group1.head(2)

,dep,nomdep,codecommune,nomcommune,prixbien,surface,surfaceterrain,tmp,KP0_prim2_P1
0,01,AIN,01001,ABERGEMENT-CLEMENCIAT,5190811.33,2960,29249,28,1753.652476
1,01,AIN,01002,ABERGEMENT-DE-VAREY,2442260.00,2091,7040,21,1167.986609


In [60]:
df_group1 = df_group1.rename(columns={
    'prixbien': 'KP0_Pri_P1',
    'surface': 'KP0_Surf_P1',
    'surfaceterrain': 'KP0_SurfTer_P1',
    'tmp' : 'KP0_transac_P1'
})
df_group1.head(2)

,dep,nomdep,codecommune,nomcommune,KP0_Pri_P1,KP0_Surf_P1,KP0_SurfTer_P1,KP0_transac_P1,KP0_prim2_P1
0,01,AIN,01001,ABERGEMENT-CLEMENCIAT,5190811.33,2960,29249,28,1753.652476
1,01,AIN,01002,ABERGEMENT-DE-VAREY,2442260.00,2091,7040,21,1167.986609


In [61]:
df_group2 = df2.groupby(['dep', 'nomdep', 'codecommune', 'nomcommune']).agg({
    'prixbien': 'sum',
    'surface': 'sum',
    'surfaceterrain': 'sum',
    'tmp' : 'count'
}).reset_index()

# PRIX / m2
df_group2['KP0_prim2_P2'] = df_group2['prixbien'] / df_group2['surface']

# RENAME
df_group2 = df_group2.rename(columns={
    'prixbien': 'KP0_Pri_P2',
    'surface': 'KP0_Surf_P2',
    'surfaceterrain': 'KP0_SurfTer_P2',
    'tmp' : 'KP0_transac_P2'
})

df_group2.head(2)

,dep,nomdep,codecommune,nomcommune,KP0_Pri_P2,KP0_Surf_P2,KP0_SurfTer_P2,KP0_transac_P2,KP0_prim2_P2
0,01,AIN,01001,ABERGEMENT-CLEMENCIAT,5396090.0,2831,19871,24,1906.072059
1,01,AIN,01002,ABERGEMENT-DE-VAREY,1885119.0,1515,5858,15,1244.302970


In [62]:
df_result = pd.merge(df_group1, df_group2, on=['dep', 'nomdep', 'codecommune', 'nomcommune'], how='outer')

# Fill NaN values with 0
df_result = df_result.fillna(0)


In [63]:
df_result.head(10)

,dep,nomdep,codecommune,nomcommune,KP0_Pri_P1,KP0_Surf_P1,KP0_SurfTer_P1,KP0_transac_P1,KP0_prim2_P1,KP0_Pri_P2,KP0_Surf_P2,KP0_SurfTer_P2,KP0_transac_P2,KP0_prim2_P2
0,01,AIN,01001,ABERGEMENT-CLEMENCIAT,5.190811e+06,2960.0,29249.0,28.0,1753.652476,5.396090e+06,2831.0,19871.0,24.0,1906.072059
1,01,AIN,01002,ABERGEMENT-DE-VAREY,2.442260e+06,2091.0,7040.0,21.0,1167.986609,1.885119e+06,1515.0,5858.0,15.0,1244.302970
2,01,AIN,01004,AMBERIEU-EN-BUGEY,1.045055e+08,55448.0,171801.0,649.0,1884.748356,1.229008e+08,59595.0,163220.0,711.0,2062.267054
3,01,AIN,01005,AMBERIEUX-EN-DOMBES,1.729253e+07,9331.0,36586.0,93.0,1853.233952,1.800978e+07,8596.0,33946.0,83.0,2095.134722
4,01,AIN,01006,AMBLEON,2.240000e+05,248.0,215.0,3.0,903.225806,9.913550e+05,574.0,3738.0,5.0,1727.099303
5,01,AIN,01007,AMBRONAY,1.668570e+07,9994.0,53438.0,97.0,1669.572243,1.601855e+07,8846.0,42416.0,92.0,1810.823649
6,01,AIN,01008,AMBUTRIX,5.685510e+06,3504.0,24146.0,35.0,1622.577055,4.279500e+06,2233.0,9063.0,23.0,1916.480072
7,01,AIN,01009,ANDERT-ET-CONDON,1.135590e+06,711.0,4403.0,8.0,1597.172996,1.506180e+06,1145.0,6130.0,11.0,1315.441048
8,01,AIN,01010,ANGLEFORT,9.290015e+06,5408.0,27053.0,58.0,1717.828217,9.135827e+06,4644.0,27921.0,48.0,1967.232343
9,01,AIN,01011,APREMONT,1.776600e+06,1291.0,11083.0,13.0,1376.142525,1.077743e+06,765.0,7308.0,7.0,1408.814418


## <a name="Nan"></a>Partie 2 : Traitement des Nan&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#TOP">&uarr;</a>

In [64]:
df_result.isna().sum()

dep               0
nomdep            0
codecommune       0
nomcommune        0
KP0_Pri_P1        0
KP0_Surf_P1       0
KP0_SurfTer_P1    0
KP0_transac_P1    0
KP0_prim2_P1      0
KP0_Pri_P2        0
KP0_Surf_P2       0
KP0_SurfTer_P2    0
KP0_transac_P2    0
KP0_prim2_P2      0
dtype: int64

In [65]:
df_result.shape

(34590, 14)

In [66]:
df_result['codecommune'].nunique()

34590

In [67]:
file_path = '../DATA/list_codecommune_leg2022_50inscrits.txt'

# Open the text file and read lines into a list
with open(file_path, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file.readlines()]

# Print or manipulate the list of lines as needed
print(lines[0:10], len(lines))

['01001', '01002', '01004', '01005', '01006', '01007', '01008', '01009', '01010', '01011'] 33830


In [68]:
codecommune_listdf_result = df_result['codecommune'].tolist()
codecommune_listdf_result[1]

'01002'

In [69]:
len(codecommune_listdf_result)

34590

In [70]:
absent = [x for x in lines if x not in codecommune_listdf_result]
len (absent)

1708

In [71]:
absent[0]

'02265'

In [72]:
df_result['DBL_bug']=0

In [73]:
for code in absent :
    new_row = [0, '*', code, '*',0,0,0,0,0,0,0,0,0,0,1]
    df_result.loc[len(df_result)] = new_row

In [74]:
df_result.head(-5)

,dep,nomdep,codecommune,nomcommune,KP0_Pri_P1,KP0_Surf_P1,KP0_SurfTer_P1,KP0_transac_P1,KP0_prim2_P1,KP0_Pri_P2,KP0_Surf_P2,KP0_SurfTer_P2,KP0_transac_P2,KP0_prim2_P2,DBL_bug
0,01,AIN,01001,ABERGEMENT-CLEMENCIAT,5.190811e+06,2960.0,29249.0,28.0,1753.652476,5.396090e+06,2831.0,19871.0,24.0,1906.072059,0
1,01,AIN,01002,ABERGEMENT-DE-VAREY,2.442260e+06,2091.0,7040.0,21.0,1167.986609,1.885119e+06,1515.0,5858.0,15.0,1244.302970,0
2,01,AIN,01004,AMBERIEU-EN-BUGEY,1.045055e+08,55448.0,171801.0,649.0,1884.748356,1.229008e+08,59595.0,163220.0,711.0,2062.267054,0
3,01,AIN,01005,AMBERIEUX-EN-DOMBES,1.729253e+07,9331.0,36586.0,93.0,1853.233952,1.800978e+07,8596.0,33946.0,83.0,2095.134722,0
4,01,AIN,01006,AMBLEON,2.240000e+05,248.0,215.0,3.0,903.225806,9.913550e+05,574.0,3738.0,5.0,1727.099303,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36288,0,*,80460,*,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,1
36289,0,*,80565,*,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,1
36290,0,*,81162,*,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,1
36291,0,*,81270,*,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,1


In [75]:
codecommune_listdf_result2 = df_result['codecommune'].tolist()
absent = [x for x in lines if x not in codecommune_listdf_result2]
len (absent)

0

In [76]:
df_result.shape

(36298, 15)

## <a name="TRANSCO"></a>Partie 3 : Filtres et/ou calculs&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#TOP">&uarr;</a>

In [77]:
len(lines)

33830

In [78]:
len(df_result)

36298

In [79]:
df_filtered = df_result[df_result['codecommune'].isin(lines)]
df_filtered.shape

(33830, 15)

## <a name="EXPORT"></a>Partie 4 : EXPORT&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#TOP">&uarr;</a>



In [42]:
for i,c in enumerate(df_filtered) : 
    print (i,c)

0 dep
1 nomdep
2 codecommune
3 nomcommune
4 KP0_Pri_P1
5 KP0_Surf_P1
6 KP0_SurfTer_P1
7 KP0_transac_P1
8 KP0_prim2_P1
9 KP0_Pri_P2
10 KP0_Surf_P2
11 KP0_SurfTer_P2
12 KP0_transac_P2
13 KP0_prim2_P2
14 DBL_bug


In [80]:
path_export='../DATA/EXPORT_CSV/final_KP0_capitalimmobilier.csv'

In [81]:
df_filtered.to_csv(path_export, sep=';', index=False)

print(f'DataFrame exported to: {path_export}')

DataFrame exported to: ../DATA/EXPORT_CSV/final_KP0_capitalimmobilier.csv
